In [2]:
%pylab inline
from collections import Counter
import pandas as pd
import glob
import os


Populating the interactive namespace from numpy and matplotlib


In [19]:
columns = Counter()
for f in glob.glob('../cross-species-data/SRP007412-*'):
    if 'kallisto' in f:
        continue
    columns += Counter(pd.read_table(f).columns.tolist())
    

In [20]:
pd.Series(columns).sort_values()

cb_M_2        1
br_F_2        1
kd_M_2        2
ht_M_2        2
br_M_5        2
br_M_4        2
br_M_3        2
lv_M_2        2
br_M_2        4
ts_M_2        6
cb_M_1        9
lv_F_1        9
ts_M_1        9
target_id    10
kd_F_1       10
ht_M_1       10
ht_F_1       10
cb_F_1       10
br_M_1       10
br_F_1       10
kd_M_1       10
lv_M_1       10
dtype: int64

In [21]:
SRP007412_cols_to_use = ['cb_M_1', 'lv_F_1', 'ts_M_1',
                         'target_id', 'kd_F_1', 'ht_M_1',
                         'ht_F_1', 'cb_F_1', 'br_M_1',
                         'br_F_1', 'kd_M_1', 'lv_M_1']


In [14]:
columns = Counter()
for f in glob.glob('../cross-species-data/SRP016501-*'):
    if 'kallisto' in f:
        continue
    columns += Counter(pd.read_table(f).columns.tolist())
    

In [15]:
pd.Series(columns).sort_values()

b_heart      4
target_id    5
c_skm        5
c_lung       5
c_liver      5
c_kidney     5
c_heart      5
c_colon      5
c_brain      5
b_testes     5
b_spleen     5
b_skm        5
b_lung       5
b_liver      5
b_kidney     5
b_colon      5
b_brain      5
a_testes     5
a_spleen     5
a_skm        5
a_lung       5
a_liver      5
a_kidney     5
a_heart      5
a_colon      5
a_brain      5
c_spleen     5
c_testes     5
dtype: int64